In [1]:
import random

In [2]:
blackjack = [
    # Ace is usable (Ace is 11)
    [],
    # Ace is unusable (Ace is 1)
    []
]
q_list = {
    True: {},
    False: {}
}
q = {
    True: {},
    False: {}
}

min_sum = 12
for a in [True, False]:
    for y in range(0, 10):
        blackjack[a].append([])
        for x in range(min_sum, min_sum + 10):
            if y + 1 == 1:
                y_text = 'A'
            else:
                y_text = str(y+1)
            blackjack[a][-1].append(f'{x}_{y_text}')
            q_list[a][f'{x}_{y_text}'] = {'hit': [], 'stand': []}
            q[a][f'{x}_{y_text}'] = {'hit': 0, 'stand': 0}


In [3]:
print('Ace is usable:')
for r in blackjack[0]:
    line = ''
    for c in r:
        line += c + '\t'
    print(line)

print()

print('Ace is unusable:')
for r in blackjack[1]:
    line = ''
    for c in r:
        line += c + '\t'
    print(line)


Ace is usable:
12_A	13_A	14_A	15_A	16_A	17_A	18_A	19_A	20_A	21_A	
12_2	13_2	14_2	15_2	16_2	17_2	18_2	19_2	20_2	21_2	
12_3	13_3	14_3	15_3	16_3	17_3	18_3	19_3	20_3	21_3	
12_4	13_4	14_4	15_4	16_4	17_4	18_4	19_4	20_4	21_4	
12_5	13_5	14_5	15_5	16_5	17_5	18_5	19_5	20_5	21_5	
12_6	13_6	14_6	15_6	16_6	17_6	18_6	19_6	20_6	21_6	
12_7	13_7	14_7	15_7	16_7	17_7	18_7	19_7	20_7	21_7	
12_8	13_8	14_8	15_8	16_8	17_8	18_8	19_8	20_8	21_8	
12_9	13_9	14_9	15_9	16_9	17_9	18_9	19_9	20_9	21_9	
12_10	13_10	14_10	15_10	16_10	17_10	18_10	19_10	20_10	21_10	

Ace is unusable:
12_A	13_A	14_A	15_A	16_A	17_A	18_A	19_A	20_A	21_A	
12_2	13_2	14_2	15_2	16_2	17_2	18_2	19_2	20_2	21_2	
12_3	13_3	14_3	15_3	16_3	17_3	18_3	19_3	20_3	21_3	
12_4	13_4	14_4	15_4	16_4	17_4	18_4	19_4	20_4	21_4	
12_5	13_5	14_5	15_5	16_5	17_5	18_5	19_5	20_5	21_5	
12_6	13_6	14_6	15_6	16_6	17_6	18_6	19_6	20_6	21_6	
12_7	13_7	14_7	15_7	16_7	17_7	18_7	19_7	20_7	21_7	
12_8	13_8	14_8	15_8	16_8	17_8	18_8	19_8	20_8	21_8	
12_9	13_9	14_9	15_9	16_9	17_9	18_9	19_9

In [4]:
ace_usable = False
agent_state = ''
policy_action = ''
agent_hand = []
dealer_hand = []
episode = []

DISCOUNT = .9


def draw_card():

    draw = random.randint(1, 13)
    if draw == 1:
        return 'A'
    elif draw > 10:
        return 10
    else:
        return draw
    

def init_game():

    global ace_usable, agent_state, agent_hand, dealer_hand, episode

    ace_usable = False
    agent_state = ''
    agent_hand = []
    dealer_hand = []
    episode = []

    for _ in range(2):
        dealer_hand.append(draw_card())
        agent_hand.append(draw_card())

    if 'A' in agent_hand:
        ace_usable = True
        hand_value = sum([11 if c == 'A' else c for c in agent_hand])
        if hand_value > 21:
            ace_usable = False


def epsilon_greedy_policy(epsilon=0.1):

    global agent_state, q, ace_usable

    if random.random() < epsilon or q[ace_usable][agent_state]['hit'] == q[ace_usable][agent_state]['stand']:
        return random.choice(['hit', 'stand']) 
    else:
        if q[ace_usable][agent_state]['hit'] > q[ace_usable][agent_state]['stand']:
            return 'hit'
        else:
            return 'stand'
        

def agent_turn():

    global ace_usable, agent_state, policy_action, agent_hand, dealer_hand

    if 'A' in agent_hand:
        ace_usable = True
        hand_value = sum([11 if c == 'A' else c for c in agent_hand])
        if hand_value > 21:
            ace_usable = False
            hand_value = sum([1 if c == 'A' else c for c in agent_hand])
    else:
        hand_value = sum(agent_hand)

    agent_state = f'{hand_value}_{dealer_hand[0]}'

    if hand_value < 12:
        agent_hand.append(draw_card())
        policy_action = None
    else:
        policy_action = epsilon_greedy_policy()


def dealer_turn():

    global ace_usable, dealer_hand

    hand_value = sum([11 if c == 'A' else c for c in dealer_hand])
    if hand_value > 21:
        hand_value = sum([1 if c == 'A' else c for c in dealer_hand])
    if hand_value < 17:
        dealer_hand.append(draw_card())


def update_q():

    global q, q_list

    states = list(q[True].keys())
    for s in states:

        if len(q_list[True][s]['hit']) > 0:
            q[True][s]['hit'] = sum(q_list[True][s]['hit']) / len(q_list[True][s]['hit'])
        else:
            q[True][s]['hit'] = 0

        if len(q_list[True][s]['stand']) > 0:
            q[True][s]['stand'] = sum(q_list[True][s]['stand']) / len(q_list[True][s]['stand'])
        else:
            q[True][s]['stand'] = 0

        if len(q_list[False][s]['hit']) > 0:
            q[False][s]['hit'] = sum(q_list[False][s]['hit']) / len(q_list[False][s]['hit'])
        else:
            q[False][s]['hit'] = 0

        if len(q_list[False][s]['stand']) > 0:
            q[False][s]['stand'] = sum(q_list[False][s]['stand']) / len(q_list[False][s]['stand'])
        else:
            q[False][s]['stand'] = 0


def play_game():

    global q, q_list, ace_usable, agent_state, policy_action, agent_hand, dealer_hand, episode

    init_game()
    episode_end = False
    while True:
        
        agent_turn()

        reward = 0

        if policy_action == 'hit':
            
            agent_hand.append(draw_card())

            if 'A' in agent_hand:
                ace_usable = True
                agent_hand_value = sum([11 if c == 'A' else c for c in agent_hand])
                if agent_hand_value > 21:
                    ace_usable = False
                    agent_hand_value = sum([1 if c == 'A' else c for c in agent_hand])
            else:
                agent_hand_value = sum(agent_hand)

            if agent_hand_value > 21: # Bust
                reward = -1
                episode_end = True
            
        elif policy_action == 'stand':

            episode_end = True
            
            if 'A' in agent_hand:
                ace_usable = True
                agent_hand_value = sum([11 if c == 'A' else c for c in agent_hand])
                if agent_hand_value > 21:
                    ace_usable = False
                    agent_hand_value = sum([1 if c == 'A' else c for c in agent_hand])
            else:
                agent_hand_value = sum(agent_hand)

            if 'A' in dealer_hand:
                dealer_hand_value = sum([11 if c == 'A' else c for c in dealer_hand])
                if dealer_hand_value > 21:
                    dealer_hand_value = sum([1 if c == 'A' else c for c in dealer_hand])
            else:
                dealer_hand_value = sum(dealer_hand)

            if agent_hand_value <= 21 and (dealer_hand_value > 21 or (21 - agent_hand_value) < (21 - dealer_hand_value)): # Win
                reward = 1
            elif agent_hand_value <= 21 and agent_hand_value == dealer_hand_value: # Draw
                reward = 0
            else: # Lose
                reward = -1

        if policy_action != None:
            episode.append({
                # state: s_t, ace_usable: s_t, action: a_t, reward: r_t+1
                'state': agent_state, 'ace_usable': ace_usable, 'action': policy_action, 'reward': reward
            })

        if episode_end:
            # Update Q(s, a) (Every visit)
            # print(episode)
            g = 0
            for e in episode[::-1]:
                g = (DISCOUNT * g) + e['reward']
                q_list[e['ace_usable']][e['state']][e['action']].append(g)
            update_q()
            break

        dealer_turn()

In [5]:
for _ in range(100000):
    play_game()

In [6]:
print('Ace is usable v_pi(s):')
for r in blackjack[0]:
    line = ''
    for c in r:
        line += c + '\t'
    print(line)
    line = ''
    for s in r:
        line += str(round(max(q[True][s]['hit'], q[True][s]['stand']), 3)) + '\t'
    print(line)

print()

print('Ace is usable v_pi(s):')
for r in blackjack[0]:
    # line = ''
    # for c in r:
    #     line += c + '\t'
    # print(line)
    line = ''
    for s in r:
        if q[True][s]['stand'] > q[True][s]['hit']:
            line += 'stand' + '\t'
        else:
            line += 'hit' + '\t'
    print(line)

print()

print('Ace is not unusable v_pi(s):')
for r in blackjack[1]:
    line = ''
    for c in r:
        line += c + '\t'
    print(line)
    line = ''
    for s in r:
        line += str(round(max(q[False][s]['hit'], q[False][s]['stand']), 3)) + '\t'
    print(line)

print()

print('Ace is not usable v_pi(s):')
for r in blackjack[0]:
    # line = ''
    # for c in r:
    #     line += c + '\t'
    # print(line)
    line = ''
    for s in r:
        if q[False][s]['stand'] > q[False][s]['hit']:
            line += 'stand' + '\t'
        else:
            line += 'hit' + '\t'
    print(line)

Ace is usable v_pi(s):
12_A	13_A	14_A	15_A	16_A	17_A	18_A	19_A	20_A	21_A	
0	-0.135	-0.225	0.02	0.0	0.312	-0.093	0.098	0.087	0.68	
12_2	13_2	14_2	15_2	16_2	17_2	18_2	19_2	20_2	21_2	
0	0.931	1.0	0.979	0.934	0.91	0.904	0.947	0.984	0.992	
12_3	13_3	14_3	15_3	16_3	17_3	18_3	19_3	20_3	21_3	
0	0.62	0.964	1.0	0.935	0.896	0.9	0.947	0.927	0.995	
12_4	13_4	14_4	15_4	16_4	17_4	18_4	19_4	20_4	21_4	
0	0.392	0.623	0.913	0.89	0.9	0.856	0.901	0.946	0.99	
12_5	13_5	14_5	15_5	16_5	17_5	18_5	19_5	20_5	21_5	
0	0.152	0.221	0.549	0.735	0.787	0.78	0.846	0.876	0.994	
12_6	13_6	14_6	15_6	16_6	17_6	18_6	19_6	20_6	21_6	
0	0.377	0.347	0.573	0.51	0.729	0.815	0.86	0.965	0.991	
12_7	13_7	14_7	15_7	16_7	17_7	18_7	19_7	20_7	21_7	
0	0.256	0.12	0.417	-0.043	0.424	0.772	0.9	0.918	0.99	
12_8	13_8	14_8	15_8	16_8	17_8	18_8	19_8	20_8	21_8	
0	0.333	0.23	0.372	0.573	0.751	0.9	0.848	0.871	0.989	
12_9	13_9	14_9	15_9	16_9	17_9	18_9	19_9	20_9	21_9	
0	0.199	0.352	0.284	0.387	0.6	0.716	0.445	0.867	0.993	
12_10	13_10	14_10	15_10	16_10

In [7]:
ace_usable = False
agent_state = ''
policy_action = ''
agent_hand = []
dealer_hand = []
episode = []

q = {
    True: {},
    False: {}
}
c = {
    True: {},
    False: {}
}

min_sum = 12
for a in [True, False]:
    for y in range(0, 10):
        for x in range(min_sum, min_sum + 10):
            if y + 1 == 1:
                y_text = 'A'
            else:
                y_text = str(y+1)
            c[a][f'{x}_{y_text}'] = {'hit': 0, 'stand': 0}
            q[a][f'{x}_{y_text}'] = {'hit': 0, 'stand': 0}


def random_action():

    global agent_state, ace_usable

    return random.choice(['hit', 'stand'])


def agent_turn():

    global ace_usable, agent_state, policy_action, agent_hand, dealer_hand

    if 'A' in agent_hand:
        ace_usable = True
        hand_value = sum([11 if c == 'A' else c for c in agent_hand])
        if hand_value > 21:
            ace_usable = False
            hand_value = sum([1 if c == 'A' else c for c in agent_hand])
    else:
        hand_value = sum(agent_hand)

    agent_state = f'{hand_value}_{dealer_hand[0]}'

    if hand_value < 12:
        agent_hand.append(draw_card())
        policy_action = None
    else:
        policy_action = random_action()


def play_game():

    global q, c, ace_usable, agent_state, policy_action, agent_hand, dealer_hand, episode

    init_game()
    episode_end = False
    while True:
        
        agent_turn()

        reward = 0

        if policy_action == 'hit':
            
            agent_hand.append(draw_card())

            if 'A' in agent_hand:
                ace_usable = True
                agent_hand_value = sum([11 if c == 'A' else c for c in agent_hand])
                if agent_hand_value > 21:
                    ace_usable = False
                    agent_hand_value = sum([1 if c == 'A' else c for c in agent_hand])
            else:
                agent_hand_value = sum(agent_hand)

            if agent_hand_value > 21: # Bust
                reward = -1
                episode_end = True
            
        elif policy_action == 'stand':

            episode_end = True
            
            if 'A' in agent_hand:
                ace_usable = True
                agent_hand_value = sum([11 if c == 'A' else c for c in agent_hand])
                if agent_hand_value > 21:
                    ace_usable = False
                    agent_hand_value = sum([1 if c == 'A' else c for c in agent_hand])
            else:
                agent_hand_value = sum(agent_hand)

            if 'A' in dealer_hand:
                dealer_hand_value = sum([11 if c == 'A' else c for c in dealer_hand])
                if dealer_hand_value > 21:
                    dealer_hand_value = sum([1 if c == 'A' else c for c in dealer_hand])
            else:
                dealer_hand_value = sum(dealer_hand)

            if agent_hand_value <= 21 and (dealer_hand_value > 21 or (21 - agent_hand_value) < (21 - dealer_hand_value)): # Win
                reward = 1
            elif agent_hand_value <= 21 and agent_hand_value == dealer_hand_value: # Draw
                reward = 0
            else: # Lose
                reward = -1

        if policy_action != None:
            episode.append({
                # state: s_t, ace_usable: s_t, action: a_t, reward: r_t+1
                'state': agent_state, 'ace_usable': ace_usable, 'action': policy_action, 'reward': reward
            })

        if episode_end:
            # Update Q(s, a) (Every visit)
            # print(episode)
            g = 0
            w = 1
            # w is our sampling weight to show how how policy b performs in relation to our policy pi
            # c is the cumalative sampling weight for each state-action pair, this tells us how much total weight has been assigned to the state-action pair across all episodes encountered so far
            # Time steps earlier in the episode are given higher sampling weights since they are what led to later state actions pairs
            # w/c(s, a) incidates how much current episode's experience for a specific state-action pair should be weighted relative to all episodes experienced so far
            for e in episode[::-1]:
                g = (DISCOUNT * g) + e['reward']
                c[e['ace_usable']][e['state']][e['action']] += w
                # q[s][a] = q[s][a] + (w/c[s][a])(g - q[s][a])
                # This is update_q() vvv
                q[e['ace_usable']][e['state']][e['action']] += (w / c[e['ace_usable']][e['state']][e['action']]) * (g - q[e['ace_usable']][e['state']][e['action']])
                # This code is for weighted importance sampling. If we were to do normal importance sampling, we replace c(s, a) with a count of how many times this state has been encountered across all episodes so far
                # w = w/b[a][s], b is random action ('hit'/'stand' are 50/50)
                w /= .5
            episode = []
            break

        dealer_turn()

In [8]:
for _ in range(100000):
    play_game()

In [9]:
print('Ace is usable v_pi(s):')
for r in blackjack[0]:
    line = ''
    for c in r:
        line += c + '\t'
    print(line)
    line = ''
    for s in r:
        line += str(round(max(q[True][s]['hit'], q[True][s]['stand']), 3)) + '\t'
    print(line)

print()

print('Ace is usable v_pi(s):')
for r in blackjack[0]:
    # line = ''
    # for c in r:
    #     line += c + '\t'
    # print(line)
    line = ''
    for s in r:
        if q[True][s]['stand'] > q[True][s]['hit']:
            line += 'stand' + '\t'
        else:
            line += 'hit' + '\t'
    print(line)

print()

print('Ace is not unusable v_pi(s):')
for r in blackjack[1]:
    line = ''
    for c in r:
        line += c + '\t'
    print(line)
    line = ''
    for s in r:
        line += str(round(max(q[False][s]['hit'], q[False][s]['stand']), 3)) + '\t'
    print(line)

print()

print('Ace is not usable v_pi(s):')
for r in blackjack[0]:
    # line = ''
    # for c in r:
    #     line += c + '\t'
    # print(line)
    line = ''
    for s in r:
        if q[False][s]['stand'] > q[False][s]['hit']:
            line += 'stand' + '\t'
        else:
            line += 'hit' + '\t'
    print(line)

Ace is usable v_pi(s):
12_A	13_A	14_A	15_A	16_A	17_A	18_A	19_A	20_A	21_A	
0	-0.207	-0.285	0.174	0.03	-0.053	-0.078	0.219	0.016	0.707	
12_2	13_2	14_2	15_2	16_2	17_2	18_2	19_2	20_2	21_2	
0	0.929	1.0	0.939	0.979	0.786	0.81	0.889	0.962	0.982	
12_3	13_3	14_3	15_3	16_3	17_3	18_3	19_3	20_3	21_3	
0	0.5	0.939	0.925	1.0	0.908	0.848	0.897	0.984	0.981	
12_4	13_4	14_4	15_4	16_4	17_4	18_4	19_4	20_4	21_4	
0	0.3	0.405	0.82	0.94	0.873	0.893	0.913	0.985	0.984	
12_5	13_5	14_5	15_5	16_5	17_5	18_5	19_5	20_5	21_5	
0	0.034	0.146	0.524	0.761	0.653	0.742	0.921	0.921	0.995	
12_6	13_6	14_6	15_6	16_6	17_6	18_6	19_6	20_6	21_6	
0	0.324	0.012	0.171	0.48	0.895	0.904	0.907	0.974	0.982	
12_7	13_7	14_7	15_7	16_7	17_7	18_7	19_7	20_7	21_7	
0	-0.113	0.031	0.238	-0.053	0.553	0.793	0.833	0.895	0.982	
12_8	13_8	14_8	15_8	16_8	17_8	18_8	19_8	20_8	21_8	
0	-0.109	-0.109	0.268	-0.157	0.056	0.365	0.797	0.986	0.995	
12_9	13_9	14_9	15_9	16_9	17_9	18_9	19_9	20_9	21_9	
0	-0.282	-0.184	-0.227	-0.109	-0.026	0.135	0.61	0.92	0.996	
12_10	